1. find the correct expression of the normalization constant in Eq. (8)

In [78]:
import numpy as np
import scipy
import sys
import scipy.special
from math import factorial
from scipy.special import hermite
import matplotlib.pyplot as plt

def norm_constant(n, nu):
    '''
    this function returns the normalisation constant for the n-th order radial wavefunction 
    '''
    denom = 4*np.pi**(3/2) * 2**(2*n+1) / (2*nu)**0.5 * factorial(2*n+1)
    return (2 / denom)**(1/2) #the factor sqrt 2 comes from hermites being even, and our given normalisation being between -inf and +inf

def R_n(n,r,nu):
    '''
    this function returns the n-th order radial wf of eigenstates of 3d HO
    '''
    return norm_constant(n,nu)* np.exp(-nu*r**2)* hermite(2*n+1)((2*nu)**0.5*r) / r

Check normalisation, change n1 and n2 to see if the radial wf's are orthonormal

In [79]:
n1 = 5
n2 = 5
nu = 1

r = np.linspace(0.1, 10, 1000)

R1 = [R_n(n1, r, nu) for r in r]
R2 = [R_n(n2, r, nu) for r in r]

integrand = [4*np.pi*r1 * r2 * r**2 for r1, r2, r in zip(R1, R2, r)]

norm = np.trapz(integrand, r)
print("The scalar product between the",n1, "wavefunction and the", n2, "wavefunction is",norm)

The scalar product between the 5 wavefunction and the 5 wavefunction is 0.9947270635579459


Matrix elements of the hamiltonian

In [ ]:
dim = 10
H = np.zeros[dim,dim]

for i in range(dim):
    for j in range(dim):
        H
        

